### Table of content<a class="anchor" id="table"></a>
* [1 - Data](#chapter1)
    * [1.1 - Data loading](#section_1_1)
    * [1.2 - Handling mini-batches](#section_1_2)
* [2 - Pieces of a Neural Network](#chapter2)
    * [2.1 - The Forward Propagation](#fwd_prop)
    * [2.2 - The Cost function](#cost_fct)
    * [2.3 - The Backward Propagation](#backprop)
    * [2.4 - Implementing the gradient descent algorithm](#gda)
* [3 - Assembling the pieces](#pieces)
    * [3.1 - Going forward](#going_fwd)
    * [3.2 - Going backward](#going_bwd)
    * [3.3 - Initializations](#init)
    * [3.4 - Prediction](#prediction)
* [4 - Training the model](#training)
    * [4.1 - The training function](#training_fct)
    * [4.2 - Preprocessing the input features](#preprocess)
    * [4.3 - Finally : training the model](#training_fin)
    * [4.4 - Discussion](#discussion)
* [5 - Improvement](#improv)
    * [5.1 - Regularization](#regularization)
        * [5.1.1 - L2 Regularization](#L2)
        * [5.1.2 - Dropout Regularization](#dropout)
    * [5.2 - Optimization](#optim)
        * [5.2.1 - Mini-Batch Gradient Descent](#minibatch)
        * [5.2.2 - Adding Momentum](#momentum)
        * [5.2.3 - Adam optimization](#adam)
        * [5.2.4 - Batch Normalization](#BN)
* [6 - Conclusion](#conclusion)

# NumPy Binary Classifier 

In this notebook, I would like to show you what is the main logic behind the common deep learning frameworks. I found this interesting since I usually tweet the hyperparameters framework's functions without really knowing what is changing into the equations.

So, a matrix point of view gives a better overview of *how a Neural Network works*, and once you're familiar with those notions, you will be able to construct deeper and more complex N.N. with the help of deep learning frameworks, knowing what you are doing. Also, it will help you to tune your model, once it has been trained, to improve it.

The goal of this classifier is to detect whether an image is a Pikachu or a Jigglypuff (english version of Rondoudou).

## [ 1 - Data](#table) <a class="anchor" id="chapter1"></a>
First we need to work on the dataset to feed the neural network with the right dimensions, types etc. For the purpuse of the exercice, I didn't chose a large dataset, it contains :
- 98 Pikachu images (label 0)
- 76 Jigglypuff [Rondoudou] images (label 1)

So 174 images that we have to randomize and split into train and test sets. 

### [1.1 - Data loading](#table) <a class="anchor" id="section_1_1"></a>
We are going to create a function which returns train/val sets and their correspondant labels. This function should let us decide the size of the test set.

Note : we usually also construct a *validation* dataset used to tune the trained model (and ensure that the *test* dataset is independant to the data used to build the model). Here, we'll use only a test set, refers through this notebook as a *validation* or a *test* set in the same way.

In [1]:
import numpy as np
import glob
import math
from PIL import Image

In [33]:
def load_data(val_size:float=0.2, seed:int=None) -> tuple:
    """
    Converts images to arrays and returns them randomized through training and 
    validation set.

    Parameters
    ----------
    val_size : float, optional
        Part of validation set. The default is 0.2.
    seed : int, optional
        Seed for repeatability. The default is None

    Returns
    -------
    data_train : np.array of shape (nb_img, HEIGHT, WIDTH, nb_chans)
        Training set.
    label_train : np.array of shape (nb_img, 1)
        Labels of the training set.
    data_val : np.array of shape (nb_img, HEIGHT, WIDTH, nb_chans)
        Validation set.
    label_val : np.array of shape (m, 1)
        Labels of the validation set.
    classes : np.array of shape (2,)
        Classe names : Pikachu / Rondoudou. They are encode in bytes.

    """
    list_pikachu = glob.glob('data/pikachu/*')
    list_rondoudou = glob.glob('data/rondoudou/*')
    
    HEIGHT = 100
    WIDTH = 100
    CHANNEL = 3
    
    print(f"Length pikachu dataset : {len(list_pikachu)}")
    print(f"Length Rondoudou dataset : {len(list_rondoudou)}")

    classes = np.array([b'Pikachu', b'Rondoudou'])
    
    # Initialisations
    size_dataset = len(list_pikachu) + len(list_rondoudou)
    dataset_arr = np.zeros((size_dataset, HEIGHT, WIDTH, CHANNEL))
    label = np.zeros((size_dataset, 1), dtype='int')
    
    # Generating a Pikachu array-type dataset
    for k in range(len(list_pikachu)):
        with Image.open(list_pikachu[k]) as im :
            im = im.resize((HEIGHT, WIDTH), resample=Image.BICUBIC)
            im = im.convert("RGB")
        img_arr = np.array(im)
        dataset_arr[k] = img_arr
        
    # Generating a Rondoudou array type dataset
    i=0
    for k in range(len(list_pikachu), len(dataset_arr)):
        with Image.open(list_rondoudou[i]) as im2 :
            im2 = im2.resize((HEIGHT, WIDTH), resample=Image.BICUBIC)
            im2 = im2.convert("RGB")
        img_arr = np.array(im2)
        dataset_arr[k] = img_arr
        label[k] = 1
        i+=1
    
    # Randomizing
    n_samples = dataset_arr.shape[0]
    n_val = int(val_size * n_samples)
    shuffled_indices = np.random.permutation(n_samples)
    train_indices = shuffled_indices[:-n_val] 
    val_indices = shuffled_indices[-n_val:]

    data_train = dataset_arr[train_indices]
    label_train = label[train_indices]
    
    data_val = dataset_arr[val_indices]
    label_val = label[val_indices]
    
    return data_train, label_train, data_val, label_val, classes

### [1.2 - Handling mini-batches](#table) <a class="anchor" id="section_1_2"></a>
Using mini-batches is an optimization technique and permits to let gradient descent makes progress *before* finishing of processing the *entire* training set. We need to create a function capable of retriving mini-batches of size `mini_batch_size` with the corresponding labels for each image.

In [32]:
def random_mini_batches(X:np.ndarray, Y:np.ndarray, mini_batch_size:int=64, seed:int=None) -> list:
    """
    Creates a list of random minibatches from (X, Y)
    
    Parameters
    ----------
    X : np.array
        Input data, of shape (input size, number of examples)
    Y : np.array
        True "label" vector, of shape (1, number of examples)
    mini_batch_size : int, optional
        Size of the mini-batches. The default is 64
    seed : int, optional
        Seed for repeatability. The default is None

    Returns
    -------
    mini_batches : list
        List of (mini_batch_X, mini_batch_Y)
    """
    np.random.seed(seed)
    m = X.shape[1]
    mini_batches = []
    end_minibatch = 0
    
    # Shuffling X, Y
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((1, m))
    
    num_complete_minibatches = math.floor(m / mini_batch_size)
    for k in range(0, num_complete_minibatches):
        start_minibatch = k * mini_batch_size
        end_minibatch = (k+1) * mini_batch_size
        mini_batch_X = shuffled_X[:, start_minibatch : end_minibatch]
        mini_batch_Y = shuffled_Y[:, start_minibatch : end_minibatch]
        
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
        
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[:, end_minibatch : ]
        mini_batch_Y = shuffled_Y[:, end_minibatch : ]
    
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

## [2 - Pieces of a Neural Network](#table) <a class="anchor" id="chapter2"></a>
In this section, we'll develop functions that are usefull to construct a neural network of size `L` with any number of units for each layer.

### [2.1 - The Forward Propagation](#table) <a class="anchor" id="fwd_prop"></a>
A neural network adjusts each hidden-layer units weights dependantly to the previous layer and go from the **input layer** to the last **output layer**. This step is called the **forward pass**. Each layer has those following variables of interest :
- $W^{[l]}$ the weights of the $l^{th}$ layer
- $b^{[l]}$ the bias of the $l^{[th]}$ layer
- $Z^{[l]}$ the pre-activation value of the $l^{[th]}$ layer : it results of a linear calculus and depends of $W^{[l]}$, $b^{[l]}$, and $A^{[l-1]}$. Note that $A^{[l-1]}$ is actually the features $X$ for the first layer. 
- $A^{[l]}$ the activation value of the $l^{[th]}$ layer computes from a given activation function.


At layer `l`, each neuron is composed of two parts : a linear part which gives `z`, and an activation part to "activate" the neurone, which gives `a`. Using matrix operations to reprensent all the units for a given layer, we got the equations below :


\begin{equation}
    \begin{cases}
      Z^{[l]} = W^{[l]}A^{[l-1]} + b\\
      A^{[l]} = g(Z^{[l]})
    \end{cases}\,.
\end{equation}

Note :
- We use lower case to talk about a neuron (or a unit) and capital case to represent a layer of neurons.
- `g` represents a given activation function for this neuron (e.g. sigmoid activation function).

In [4]:
def activation_function(Z:np.ndarray, activation:str) -> np.ndarray:
    """
    Calculate the activation function

    Parameters
    ----------
    Z : np.array
        pre-activation parameter.
    activation : str
        Name of the activation function (sigmoid or relu)
    Returns
    -------
    A : np.array of shape Z.shape
        Post-activation parameter

    """
    A = np.zeros(Z.shape)
    if activation == 'sigmoid':
        A = 1 / (1 + np.exp(-Z))
    elif activation == 'relu':
        A = np.maximum(0,Z) 
    return A


def forward_pass(A_prev:np.ndarray, W:np.ndarray, b:np.ndarray, activation:str) -> tuple:
    """
    Implement the forward pass
    LINEAR->ACTIVATION layer

    Parameters
    ----------
    A : np.array of shape (n[l-1], m)
        Activations from previous layer (or input data).
    W : np.array of shape (n[l], n[l-1])
        Weights matrix.
    b : np.array of size (n[l], 1)
        Bias vector.
    activation : str
        Name of activation function.

    Returns
    -------
    A : np.array
        Post-activation value.
    cache : tuple
        containing "linear_cache" and "activation_cache". Storing variable for 
        computing the backward pass efficiently.

    """
    Z = np.dot(W, A_prev) + b
    A = activation_function(Z, activation)

    linear_cache = (A_prev, W, b)
    activation_cache = Z
    cache = (linear_cache, activation_cache)
    return A, cache

### [2.2 - The Cost function](#table) <a class="anchor" id="cost_fct"></a>
The cost function computes how much the predicted result (at the output layer) is far from a given ground truth. We'll define *cost function* `J` as the sum of the loss functions `L` over all the examples (from 1 to m). For logistic regression, we usually use the *cross entropy cost function* (defined above). 

In machine learning, we want to *minimise* the cost function *i.e.* find the best configuration of weights en bias who provide a prediction that makes the cost function tends toward zero. A well known algorithm to optimize parameters is the *gradient descent algorithm* : it computes new weights and bias for each iteration based on how much the cost function `J` varies and it is scaled by a `learning rate` $\alpha$. 

\begin{equation}
    \begin{cases}
    W = W - \alpha \frac{\partial J}{\partial W} \\
    b = b - \alpha \frac{\partial J}{\partial b}
    \end{cases}\,.
\end{equation}

\begin{equation}
    J(W,b) = \frac{1}{m}\sum_{i=1}^{m}L(\hat{Y}, Y) =  \frac{-1}{m}\sum_{i=1}^{m}\left(Y\log(\hat{Y}) + (1-Y)\log(1-\hat{Y})\right)
\end{equation}


*Note : since the gradient descent is applied only on a portion of the data when using mini-batch, we do not normalized by the number of examples ($\frac{1}{m}$ factor) when dealing with mini-batches.*

In [5]:
def compute_cost(yhat:np.ndarray, Y:np.ndarray, mini_batch_size:int=None) -> float:
    """
    Compute the cost function, sum of the loss function

    Parameters
    ----------
    yhat : np.array of shape (1, m)
        Probability vector corresponding to the label predictions. It's actually
        the activation at the layer L : AL.
    Y : np.array of shape (1, m)
        True label vector.
    mini_batch_size : int, optionnal
        Arg used to trigger the normalization of the cost by 1/m if there is no
        mini_batches. Default is None.

    Returns
    -------
    cost : float
        Cross-entropy cost function with or without dividing by number of 
        training examples

    """
    AL = yhat
    cost = np.sum(-np.log(AL)*Y - np.log(1-AL)*(1-Y))
    
    if mini_batch_size is None:
        m = Y.shape[1]
        cost = (1/m) * cost

    return cost


### [2.3 - The Back Propagation](#table) <a class="anchor" id="backprop"></a>
We saw earlier how the gradient descent algorithm works : \
\begin{equation}
    \begin{cases}
    W = W - \alpha \frac{\partial J}{\partial W} \\
    b = b - \alpha \frac{\partial J}{\partial b}
    \end{cases}\,.
\end{equation}

The goal of the back propagation is to calculate the gradients of the cost function with respect to the parameters. Knowing the formula of $J$, we can understand that it will be a little bit tricky to get those gradients. 

We recall that $\hat{Y}$ is the result of the forward pass which involved one linear part $Z^{[l]}(W^{[l]},A^{[l-1]}, b)$ and a non-linear part $A^{[l]}(Z^{[l]})$. So, $J$ is a function of functions. I won't go into calculus details, but if you do the maths, you'll get to these formulas :

$$ dZ^{[l]} = \frac{\partial J }{\partial Z^{[l]}} = dA^{[l]} g'(Z^{[l]}) $$
$$ dW^{[l]} = \frac{\partial J }{\partial W^{[l]}} = \frac{1}{m} dZ^{[l]} A^{[l-1] T}$$
$$ db^{[l]} = \frac{\partial J }{\partial b^{[l]}} = \frac{1}{m} \sum_{i = 1}^{m} dZ^{[l]}$$
$$ dA^{[l-1]} = \frac{\partial J }{\partial A^{[l-1]}} = W^{[l] T} dZ^{[l]}$$

In [6]:
def backward_activation_function(dA:np.ndarray, cache:np.ndarray, activation:str) -> np.ndarray:
    """
    Implements the backward propagation for SIGMOID or ReLU. 
    Compute dZ

    Parameters
    ----------
    dA : np.array of any shape
        Post-activation gradient
    cache : np.array of shape of dA
        Used to compute backward propagation efficiently.
    activation : str
        Name of the activation function

    Returns
    -------
    dZ : np.array of shape dA.shape
        Gradient of the cost with respect to Z.

    """
    Z = cache
    dZ = np.zeros(Z.shape)
    if activation == 'sigmoid':
        s = activation_function(Z, activation)
        dZ = dA * s * (1 - s)
    elif activation == 'relu':
        dZ = np.array(dA, copy=True)
        dZ[Z <= 0] = 0
    return dZ


def backward_pass(dA:np.ndarray, cache:tuple, activation:str) -> tuple:
    """
    Compute the backward pass :
    LINEAR -> dZ -> GRADS

    Parameters
    ----------
    dA : np.array
         Post-activation gradient for current layer l.
    cache : tuple
        containing "linear_cache" and "activation_cache". These caches come from 
        the forward pass.
    activation : str
        Activation to be used in this layer, stored as a text string: "sigmoid" or "relu".

    Returns
    -------
    dA_prev : np.array of shape A.shape
        Gradient of the cost with respect to the activation (previous layer l-1)
    dW : np.array of shape W.shape
        Gradient of the cost with respect to the activation (current layer)
    db : np.array of shape b.shape
        Gradient of the cost with respect to b (current layer l).

    """
    linear_cache, activation_cache = cache
    A_prev, W, b = linear_cache
    m = A_prev.shape[1]
    dZ = np.zeros(dA.shape)
    
    if activation == 'relu':
        dZ = backward_activation_function(dA, activation_cache, 'relu')
    elif activation == 'sigmoid':
        dZ = backward_activation_function(dA, activation_cache, 'sigmoid')
    
    ### Compute grads
    dW = (1./m) * np.dot(dZ, A_prev.T)
    db = (1./m) * np.sum(dZ, axis=1, keepdims=True)
    dA_prev =  np.dot(W.T, dZ)

    return dA_prev, dW, db

### [2.4 - Implementing the gradient descent algorithm](#table) <a class="anchor" id="gda"></a>
Let's built a function that updates parameters with the corresponding gradient at a rate set by the `learning_rate` hyperparameter.

In [7]:
def update_parameters(parameters:dict, grads:dict, learning_rate:float) -> dict:
    """
    Update parameters using gradient descent

    Parameters
    ----------
    params : dict
        Containing the parameters.
    grads : dict
        Contain the grads output of model_backward.
    learning_rate : float
        Tune the rate of learning.

    Returns
    -------
    parameters : dict
        Contain the update parameters.

    """
    L = len(parameters) // 2
    
    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - \
            learning_rate * grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - \
            learning_rate * grads["db" + str(l+1)]
    
    return parameters

## [3 - Assembling the pieces](#table) <a class="anchor" id="pieces"></a>
Until now, we've build functions that will help us to proceed to the forward pass and the backward pass for any layers. Let's now assemble some pieces.

### [3.1 - Going forward](#table) <a class="anchor" id="going_fwd"></a>
The function below computes the forward pass through the layers from the first to the output layer. For network stability, we often use the **ReLU** activation function to activate the neurons *inside* the network and then, prevent gradients to vanish. Since we need a binary output as a probability of prediction, the output layer will be activated by a **sigmoid** activation function. 

*Note : the hyperparameter `keep_prob` is used to regularize the model by **Dropout**, see the [Dropout Regularization](#dropout) part of this notebook.* 

In [40]:
def forward(X:np.ndarray, parameters:dict, keep_prob:float=1) -> tuple :
    """
    Implement forward propagation for a single layer (layer l) with ReLU activation function inside
    the network and Sigmoid activation function for the output layer.

    Parameters
    ----------
    X : np.array
        Data input.
    parameters : dict
        Parameters of the current layer
    keep_prob : float, optional
        Dropout factor : trigger the dropout regularization if < 1. The default is 1.

    Returns
    -------
    AL : np.array
        Activation value from the output (last) layer, i.e. prediction vector, also called yhat.
    caches : list
        Contain every cache of the linear part of the forward propagation : there are L of them, 
        indexed from 1 to L.
    listD : list
        list containing every mask used to dropout the layer [l] : there are L-1 of them, 
        indexed from 1 to L-1.

    """
    caches = []
    A = X
    L = len(parameters) // 2
    listD = []
        
    ### Hidden Layers
    for l in range(1, L):
        A_prev = A
        W = parameters["W"+str(l)]
        b = parameters["b"+str(l)]
        A, cache = forward_pass(A_prev, W, b, activation="relu")
        caches.append(cache)
        
        # Handle dropout regularization
        if keep_prob < 1:
            D = np.random.rand(A.shape[0], A.shape[1])
            D = (D<keep_prob).astype(int) # mask creation
            A = A * D
            A = A / keep_prob
            listD.append(D)
    
    ### Output layer
    A_prevL = A
    l = L-1
    WL = parameters["W"+str(l+1)]
    bL = parameters["b"+str(l+1)]
    AL, cache = forward_pass(A_prevL, WL, bL, activation = "sigmoid")
    caches.append(cache)
    
    return AL, caches, listD


### [3.2 - Going backward](#table) <a class="anchor" id="going_bwd"></a>
The function below computes the backward pass through the layers from the output layer to the first hidden layer. 
So, like a reverse of the forward pass, we start to compute the gradients of the sigmoid activation function at the output layer (layer L) and then going backward until the first hidden layer : from layer L-1 to layer 1, using $dA^{[L-1]}$ to get $dA^{[L-2]}$... until getting $dA^{[0]}$.

- *Note : again, the hyperparameter `keep_prob` and the parameter `listD` are used to regularize the model by **Dropout Regularization**, see the [Dropout Regularization](#dropout) part of this notebook.* 
- *Note 2 : the hyperparameter `lambd` is used to regularize the model by **L2 Regularization**, see the [L2 Regularization](#L2) part of this notebook.*

In [31]:
def backward(AL:np.ndarray, Y:np.ndarray, caches:list, lambd:float=0, listD:list=[], keep_prob:float=1.) -> dict:
    """
    Implement the backward propagation for the 
    [LINEAR->RELU] * (L-1) -> LINEAR -> SIGMOID group.
    Can use the L2 and/or dropout regularizations.

    Parameters
    ----------
    AL : np.array of shape Y.shape
        Probability vector, output of the forward propagation (L_model_forward()).
        Also called yhat.
    Y : np.array
        True "label" vector (containing 0 or 1).
    caches : list
        Contain: every cache of linear_activation_forward() with "relu" 
        (it's caches[l], for l in range(L-1) i.e l = 0...L-2) and the cache 
         of linear_activation_forward() with "sigmoid" (it's caches[L-1]).
    lambd : float
        L2 regularization factor : trigger the L2reg if > 0. The default is 0.
    listD : list
        list containing every mask used to dropout the layer [l]. 
        (there are L-1 of them, indexed from 1 to L-1).
    keep_prob : float, optional
        Dropout factor : trigger the dropout regularization if < 1. The default is 1.
    
    Returns
    -------
    grads : dict
        A dictionary with the gradients.

    """
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    
    dAL =  - np.divide(Y, AL) + np.divide(1 - Y, 1 - AL)
    
    ### Output layer L (using dAL to get dAprev i.e. dA[L-1])
    current_cache = caches[L-1]
    dA_prev_temp, dW_temp, db_temp = backward_pass(dAL, current_cache, 'sigmoid')
    
    ### L2 regularization terms initialization
    L2reg_dW = np.array([0.])
    L2reg_db = np.array([0.])
    
    ### Handle L2 regularization for output layer
    if lambd > 0 :
        L2reg_dW = (lambd/m) * dW_temp
        L2reg_db = (lambd/m) * db_temp
    
    grads["dA" + str(L-1)] = dA_prev_temp
    grads["dW" + str(L)] = dW_temp + L2reg_dW
    grads["db" + str(L)] = db_temp + L2reg_db
    
    ### Hidden layers : from layer L-1 to layer 1 using dA[L-1] to get dA[L-2]... until dA[0]
    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = backward_pass(grads["dA" + str(l+1)], current_cache, 'relu')
        
        # Handle dropout 
        if keep_prob < 1:
            dA_prev_temp = dA_prev_temp * listD[l]
            dA_prev_temp = dA_prev_temp / keep_prob
        
        # Handle L2 regularization for hidden layers
        if lambd > 0:
            L2reg_dW = (lambd/m) * dW_temp
            L2reg_db = (lambd/m) * db_temp
            
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp + L2reg_dW
        grads["db" + str(l + 1)] = db_temp + L2reg_db
        
    return grads


### [3.3 - Initializations](#table) <a class="anchor" id="init"></a>
The first layer's weights are initialized randomly to "initiate the learning". Here we'll use the *He-et-al Initialization* technique since we are using ReLU activation. This method initiates weights randomly and scales them regarding the size of the previous layer :

$$W^{[0]} = \frac{W_{rand}}{\sqrt{[l^{-1}]}}$$

Where :
- $W_{rand}$ is a randomize matrice of weights
- $[l^{-1}]$ is the size of the layer $l^{-1}$

*Note : one could demonstrates that initialized the biais b randomly is useless and doesn't impact the learning of the network. So usually, it is sets to zero with the right dimensions.*

In [34]:
def initialize_parameters(layer_dims:list, seed:int=None) -> dict:
    """
    Initialize randomly parameters W and b of each layer

    Parameters
    ----------
    layer_dims : list[np.array]
        Containing the dimensions of each layer in the network.
    seed : int, optional
        Seed for repeatability. The default is None

    Returns
    -------
    parameters : dict
        Contain parameters "W1", "b1", ..., "WL", "bL".

    """
    parameters = {}
    L = len(layer_dims)
    
    for l in range(1, L):
        parameters["W" + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) / np.sqrt(layer_dims[l-1])
        parameters["b" + str(l)] = np.zeros((layer_dims[l],1))
    
    return parameters

### [3.4 - Prediction](#table) <a class="anchor" id="prediction"></a>

The prediction function is used to get a prediction when it has been decided that the parameter's model are well trained, *i.e* that the cost function has decreased enough. This function is just the forward part of the model, using the optimized parameters `W`and `b` get in the training part, and converts the probability prediction into binary label 0 or 1. It gives also the **accuracy** of the model as the sum of the predicting labels regarding the true labels :

$$ accuracy = \frac{1}{m}\sum^{m}_{1}(\hat{y} == y)$$

In [35]:
def predict(X:np.ndarray, y:np.ndarray, parameters:dict, training_set:bool=True) -> float:
    """
    Function used to predict the results of a  L-layer neural network.

    Parameters
    ----------
    X : np.array
        Input dataset.
    y : np.array
        True "label" vector (containing 0 or 1).
    parameters : dict
        Parameters of the trained model.
    training_set : bool, optional
        Verbose parameter

    Returns
    -------
    p : int
        Prediction label (0 or 1).

    """
    if training_set : 
        name = "Training"
    else:
        name = "Validation"
    
    m = X.shape[1]
    p = np.zeros((1,m))
    
    # Forward pass
    probas, caches, _ = forward(X, parameters)
        
    for i in range(0, probas.shape[1]):
        p[0,i] = round(probas[0,i])
        
    print(f"{name} Accuracy: {np.sum((p == y)/m)*100 :.2f}%")
    return np.sum((p == y)/m)


## [4 - Training the model](#table) <a class="anchor" id="training"></a>
### [4.1 - The training function](#table)<a class="anchor" id="training_fct"></a>


Now that all the functions are well defined, we can built our model using the forward pass, the backward pass etc...

Let's create the `NN_model` function with the right arguments that will be used to train the parameters `W` and `b` through `epochs` representing one whole step of calculation (forward + backward + parameters update + loss computation). 

As explained earlier, the training will handle the `mini-batch` optimization technique so, two loops will be necessary : one to go through the epochs and one to analyse each set of image stored as mini-batch. 

In [62]:
def NN_model(X:np.ndarray, Y:np.ndarray, layers_dims:list, learning_rate:float=0.0075, 
             mini_batch_size:int=64, epochs:int=1000) -> tuple:
    """
    Implements a L-layer neural network: [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID.
    With potential regularization techniques and optimization techniques.

    Parameters
    ----------
    X : np.array
        Input features.
    Y : np.array of shape (1, m)
        True "label" vector (containing 0 if Pikachu, 1 if Rondoudou)
    layers_dims : list of length (number of layers + 1)
        Contain the input size and each layer size.
    optimizer : str, optional
        Optimization technique to use. The default is gradient descent 'gd'.
    learning_rate : float, optional
        Learning rate of the gradient descent update rule. The default is 0.0075.
    mini_batch_size : int, optional
        Size of the mini_batch.
    epochs : int, optional
         Number of iterations of the optimization loop. The default is 1000.

    Returns
    -------
    parameters : dict
        Parameters learnt by the model.
    costs : list
        List of costs for each epoch.

    """
    costs = []
    t = 0
    m = X.shape[1]
    
    # Initialize parameters
    parameters = initialize_parameters(layers_dims, seed=42)
    
    
    print(f"Training is starting with minibatch size = {mini_batch_size} ...")
    for epoch in range(1, epochs+1):
        # Chose a minibatch per epoch
        minibatches = random_mini_batches(X, Y, mini_batch_size, seed=42)
        cost_total = 0
        
        # Training through minibatch
        for minibatch in minibatches:
            # Select a minibatch
            (minibatch_X, minibatch_Y) = minibatch 
        
            # Forward pass
            yhat, caches, listD = forward(minibatch_X, parameters)
            
            # Compute cost and add to the cost total
            cost = compute_cost(yhat, minibatch_Y, mini_batch_size)
            cost_total += cost
        
            # Backward pass
            grads = backward(yhat, minibatch_Y, caches)
        
            # Update parameters
            parameters = update_parameters(parameters, grads, learning_rate)
                
        # Compute average cost
        cost_avg = cost_total / m
            
        # Printing
        if epoch==1 or epoch % 100 == 0 or epoch == epochs: #- 1:
            print("{} ------- Cost after epoch {}: {:.4f}".format(
                datetime.datetime.now().strftime('%Y-%m-%d_%H.%M.%S'), 
                epoch, 
                cost_avg))
        if epoch % 100 == 0 or epoch == epochs:
            costs.append(cost_avg)
            
    return parameters, costs

### [4.2 - Preprocessing the input features](#table)<a class="anchor" id="preprocess"></a>

Let's use our `load_data` and `random_mini_batches` functions to create input features. Note that to get a faster learning, it's common use to normalize the data between 0 and 1. We'll also perform some reshapes to fit the data to the required model's input shape.

In [63]:
# Loading libraries
import numpy as np
import datetime
import glob
from PIL import Image

In [64]:
# Preprocessing
data_train_orig, label_train, data_val_orig, label_val, classes = load_data(seed=42)

m_train = data_train_orig.shape[0]
m_test = data_val_orig.shape[0]

# Reshape the training and test examples 
data_train_flatten = data_train_orig.reshape(m_train, -1).T
data_val_flatten = data_val_orig.reshape(m_test, -1).T

# Standardize data to have feature values between 0 and 1.
data_train = data_train_flatten/255.
data_val = data_val_flatten/255.

label_train = label_train.T
label_val = label_val.T

print ("\n----------------------\n")
print ("data_train's shape: " + str(data_train.shape))
print ("data_val's shape: " + str(data_val.shape))
print ("\n----------------------\n")

n_x = data_train_orig.shape[1]*data_train_orig.shape[2]*data_train_orig.shape[3]
n_y = 1

Length pikachu dataset : 98
Length Rondoudou dataset : 76

----------------------

data_train's shape: (30000, 140)
data_val's shape: (30000, 34)

----------------------



### [4.3 - Finally : training the model](#table)<a class="anchor" id="training_fin"></a>

In [65]:
# Training a 4-layers model
layers_dims = [n_x, 10, 5, 3, n_y]
parameters, costs = NN_model(data_train, 
                             label_train, 
                             layers_dims, 
                             learning_rate = 0.001,
                             epochs = 400)
pred_train = predict(data_train, label_train, parameters)

Training is starting with minibatch size = 64 ...
2022-04-25_08.48.12 ------- Cost after epoch 1: 0.6921
2022-04-25_08.48.16 ------- Cost after epoch 100: 0.5510
2022-04-25_08.48.21 ------- Cost after epoch 200: 0.4131
2022-04-25_08.48.25 ------- Cost after epoch 300: 0.3640
2022-04-25_08.48.30 ------- Cost after epoch 400: 0.3377
Training Accuracy: 97.86%


### [4.4 - Discussion](#table)<a class="anchor" id="discussion"></a>
Yeay ! The model works and it learns ! 
We have walked a long way to come here, defined functions to build our own deep neural network from scratch. Now let's talk about the results.

First, we can see that the training accuracy of this simple network is high. As we now, the training accuracy is a measure of how the network behaves facing knowing images. Let's see how it behaves with unknowing images.

In [66]:
pred_test = predict(data_val, label_val, parameters, training_set=False)

Validation Accuracy: 94.12%


Well, 94% accuracy, for a start it's interesting !

Unfortunately, things might not always be so easy : first, remember that a high training accuracy regarding validation (or test) accuracy is a strong alert of overfitting. That is, the network will learn the data too "strongly" and focuses on all the noisy and unhelpful parts, it's often the sign of an unflexible too complex model (it is too constraint to generalize well to unknown images). In that case, we talk about **overfitting** or a model with high **variance**.

Here, the model seems to generelize well to unknown images (at least, regarding the training accuracy). But relative to this easy task (cartoon images, small dataset) it should give better results (near 99%). We've noticed that the model's training is highly dependant of the random initialization and could render better (or worst) results (we set a seed here to bypass this effect). Also, we might be able to get better results by building a more complex model with more units (neurons) in each layer.


**Finaly :** 

One thing we should remember is the size of our database : it's a really small database for image recognition. The training set is so small that it isn't representative of the various Pikachu or Rondoudou images that could be found on the web. So, we have to beware of the robustness of our model (which is really weak here). \
Also, we could talk about the kind of image that we want to learn : Pikachus or Rondoudous are actually drawings, cartoon objects. So, we expect the images to be net, with small textures and colors (like hair, beard, skin color, clothing etc…). Knowing that, we are expecting the network to learn more easily and so required less input images for training. \
And last but not least, this model uses what we call "fully connected layers" : each pixel of the image are fully linked to each layer of the network by a linear function (before non-linear activation). This, leads to computationaly intensive work and a weak spatial invariance (capacity to recognize a target as the target, even when its appearance varies in some way like a translation, light variations...). Convolutional Neural Networks will fix this issues by "highlighting" region of interest in the image and sharing parameters through the network (but this feature is beyond the scope of this notebook).

**What happends next ?** \
We saw that the size of our dataset and the complexity of our model don't need tricks to speed up the learning, neither to improve it. Nevertheless, in the case of large datasets, with maybe bigger images and more complex models, tricks need to be invoke to ensure the model is generelizing well and the loss function is decreasing well enough in a fair descent running time. So, let's see some improvements.

## [5 - Improvement](#table) <a class="anchor" id="improv"></a>
In this section, we'll show the bases of the improvements that could be implement in the network to get a better control of the learning process. We will show two ways of such improvement : regularization techniques, who's going to make the network simpler and then, prevent overfitting. Optimization techniques, speeding up the learning.

### [5.1 - Regularizations](#table) <a class="anchor" id="regularization"></a>


#### [5.1.1 - L2 Regularization](#table) <a class="anchor" id="L2"></a>

L2 regularization technique or *the technique of the weight decays* introduces a `lambd` hyperparameter to control the weights of the matrices (see the [forward](#going_fwd) and [backward](#going_bwd) parts). It also adds a term in the cost formula such as :
\begin{equation}
    {cost}_{L2reg} = \frac{-1}{m}\sum_{i=1}^{m}\left(Y\log(\hat{Y}) + (1-Y)\log(1-\hat{Y})\right) + \frac{\lambda}{2m} \sum_{l=1}^{L} \left\|W^{[l]}  \right\|^{2}_{F}
\end{equation}


Where : 
- $W^{[l]}$ is the weight matrice of the layer $l$
- $\left\|.\right\|_{F}$ is the Frobenius norm
- $\lambda$ is a regularization hyperparameter $\geq$ 0

So, as $\lambda$ increase, the regularization term is shrinking the weights toward zero. The network is getting simpler and so, less prone to overfitting (in fact, it tends to the logistic regression which has high bias). 

In [ ]:
def compute_cost_L2regularization(yhat:np.ndarray, Y:np.ndarray, layers_dim:list, parameters:dict, lambd:float, mini_batch_size:int=None):
    """
    Compute the cost function, sum of the loss function, with L2 regularization.

    Parameters
    ----------
    yhat : np.array of shape (1, m)
        Probability vector corresponding to the label predictions. It's actually
        the activation at the layer L : AL.
    Y : np.array of shape (1, m)
        True "label" vector.
    layers_dims : list of length (number of layers + 1)
        Contain the input size and each layer size.
    parameters : dict
        Output of initialize_parameters_deep().
    lambd : float
        Regularization factor

    Returns
    -------
    cost : float
        Cross-entropy cost.

    """
    AL = yhat
    m = AL.shape[1]
    cross_entropy_cost = compute_cost(AL, Y, mini_batch_size)
    
    somme = 0
    for l in range(1, len(layers_dim)):
        W = parameters["W"+str(l)]
        somme = somme + np.sum(np.square(W))
    
    L2_regularization_cost = (1/m)*(lambd/2) * somme
    cost = cross_entropy_cost + L2_regularization_cost
    
    return cost

#### [5.1.2 - Dropout Regularization](#table) <a class="anchor" id="dropout"></a>
Dropout is an other regularization technique that could be implement like the L2 regularization. Rather than working on the weight matrices, it takes effect directly on a node of a layer $l$ : it goes through the network and eliminates a node with a certain probability. Using *Dropout* leads to a diminished network for each $m$ example and then, it ends up by training a much smaller network (and therefore a simpler one) for each example. 

**Implementation :**
- First, we set a dropout mask $d^{[l]}$ with the size of the actual activation layer $A^{[l]}$ such as the probability of **keeping** the node is defined by the hyperparameter $keep\_prob$ :
$$d^{[l]} = random([A^{[l]}]) < keep\_prob$$

- Then, we turn off some activation nodes, regarding the dropout mask $d$ :
$$A^{[l]} = A^{[l]} \times d^{[l]}$$ 

- Finally, even if the activation layer $A^{[l]}$ has been reduced, we still don't want to reduce the expected value of the pre-activation next layer $Z^{[l+1]}$. So, we must scaling up the reduced activation layer such as :
$$A^{[l]} = A^{[l]} / keep\_prob$$ 

This technique is actually called **Inverted Dropout Technique**.

<br>

*Note :*
- *In the [backward part](#going_bwd), each dropout mask for each layer is stored in the `listD` parameter*
- *It's common to use dropout for layers with a lot of parameters (in order to prevent potential overfitting). At least, it is common to use a large `keep_prob` for layers who should not have lot of parameters*
- *We won't use dropout at test (validation) time because we don't want the output to be random. This would just add noise to the predicitons*

### [5.2 - Optimizations](#table) <a class="anchor" id="optim"></a>

#### [5.2.1 - Mini-Batch Gradient Descent](#table) <a class="anchor" id="minibatch"></a>
As defined earlier (see [part 1.2](#section_1_2)) the mini-batch technique is a way to speeding-up the learning of the network. Indeed, usually the network processes the *entire* training set before one step of gradient descent. With mini-batches, gradient descent makes progress *before* finishing of processing the entire training set. The network must contain an extra inner loop to go through each mini-batch (see [part 4.1](#training_fct)). Also, a mini-batch cost function is computed to keep track of the variation of the loss into the mini-batch computing.

Mini-batch Gradient Descent makes a param update after seeing just a subset of examples so, the direction of the update has some variance and the path taken will "oscillate" toward convergence.

For some problems with large training set, the **Stochastic Gradient Descent** could be used by setting the mini-batch size to 1. It calculates the error and updates the model for each example in the training dataset. The increased model update frequency could result in faster learning and the noisy update can allow the model to avoid local minima. The main disavantages is losing the benefit of vectorisation (since it processes a single example at a time) and the cost doesn't converge to the real minimum because of the noise introduced by taking care of one single example at a time.

<br>

*Note :* 
- *The basic Batch Gradient Descent is usually chosen for small training set (m $\leq$ 2000 examples)*
- *The mini-batch optimization technique provides a fair improvement of time computing if the number of training examples is over 2000*
- *The mini-batch size is set as $2^{2n}$ (64, 128, 512...) to mirroring the way of how the computer memory works*
- *We must be sure that each mini-batch fits in the CPU/GPU memory*



#### [5.2.2 - Adding Momentum](#table) <a class="anchor" id="momentum"></a>

Adding momentum to the gradient descent optimization algorithm allows the search to overcome the oscillations of noisy gradients. In particular, it tends to average one direction of the oscillations to accelerate the convergence (helps to learn in strait line). Momentum can be associated as a velocity since it controls the gradients $dW$ and $db$ which are the variations of $W$ and $b$. 

Gradient Descent with momentum uses *exponentially weighted moving averages* (EWMA) statistic notions. The EWMA could be define like : 

$$ v_{\theta} = \beta v_\theta + (1-\beta) \theta_t $$

Where :
- $v_\theta$ is the average value of the parameter $\theta$ also called the "velocity" of the momentum optimization
- $\beta$ is the momentum parameter : it represents the number of 'step' taking into account for computing the average $v_\theta$ defined by the formula : $\frac{1}{1-\beta}$
- $t$ si the EWMA "step"
- $\theta_t$ is the "t" measured parameter 

Usually, we also do a "bias correction" which makes the computations of the EWMA more accurate during the inital phases : 

$$v_t >>> \frac{v_t}{1-\beta^t}$$

So, for each epoch, we have to compute $v_{dw}$ and $v_{db}$ the average parameters and we must redefine the gradient descent update parameter step :

\begin{equation}
    \begin{cases}
        v_{dw} = \beta v_{dw} + (1-\beta)dW\\
        v_{db} = \beta v_{db} + (1-\beta)db
    \end{cases}\,
\end{equation}

\begin{equation}
    \begin{cases}
        W = W - \alpha v_{dw}\\
        b = b - \alpha v_{db}\\
    \end{cases}\,
\end{equation}

<br>

A common value for momentum $\beta$ is 0.9 such as the EWMA uses arround $\frac{1}{1-0.9} = 10$ gradients to compute the averages.

In [ ]:
def initialize_velocity(parameters:dict) -> dict:
    """
    Initializes the velocity for the momentum optimization as a python 
    dictionary with:
                - keys: "dW1", "db1", ..., "dWL", "dbL" 
                - values: numpy arrays of zeros of the same shape as the 
                corresponding gradients/parameters.
    Parameters
    ----------
    parameters : dict
        Contain the weights and bias at the layer l
    
    Returns
    -------
    v : dict
        Contain the current velocity at the layer l for the corresponding gradients.
                    v['dW' + str(l)] = velocity of dWl
                    v['db' + str(l)] = velocity of dbl
    """
    L = len(parameters) // 2
    v = {}
    
    # Initialize velocity
    for l in range(1, L + 1):
        v["dW" + str(l)] = np.zeros(parameters["W" + str(l)].shape)
        v["db" + str(l)] = np.zeros(parameters["b" + str(l)].shape)        
    return v



def momentum_update_parameters(parameters:dict, grads:dict, v:dict, beta:float, learning_rate:float):
    """
    Update parameters using gradient descent

    Parameters
    ----------
    parameters : dict
        Containing the parameters.
    grads : dict
        Contain the grads output of the backward pass.
    v : dict
        Contain the current velocity at the layer l for the corresponding gradients.
                    v['dW' + str(l)] = velocity of dWl
                    v['db' + str(l)] = velocity of dbl
    beta : float
        Momentum hyperparameter
    learning_rate : float
        Tune the rate of learning.

    Returns
    -------
    parameters : dict
        Contain the update parameters.
    v : dict
        Contain the current velocity at the layer l for the corresponding gradients.
    
    """
    L = len(parameters) // 2
    
    # Momentum update for each parameter
    for l in range(1, L+1):
        v["dW" + str(l)] = beta * v["dW" + str(l)] + (1 - beta) * grads["dW" + str(l)]
        v["db" + str(l)] = beta * v["db" + str(l)] + (1 - beta) * grads["db" + str(l)] 
        
        parameters["W" + str(l)] = parameters["W" + str(l)] - \
            learning_rate * v["dW" + str(l)]
        parameters["b" + str(l)] = parameters["b" + str(l)] - \
            learning_rate * v["dW" + str(l)]
    
    return parameters, v


#### [5.2.3 - Adam optimization](#table) <a class="anchor" id="adam"></a>

**Ada**ptative **M**oment Estimation or Adam optimization technique is a combination of two optimization techniques : Momentum and RMSprop. 
RMSprop can be define like Momentum optimization : it modifies the update parameter's step with the help of the exponential weighted moving average :

\begin{equation}
    \begin{cases}
        S_{dW} = \beta + (1-\beta)dW^2\\
        S_{db} = \beta + (1-\beta)db^2
    \end{cases}
\end{equation}

\begin{equation}
    \begin{cases}
        W = W - \alpha \frac{dW}{\sqrt{S_{dW}}+\epsilon}\\
        b = b - \alpha \frac{db}{\sqrt{S_{db}}+\epsilon}
    \end{cases}
\end{equation}

Finally, Adam optimization combines Momentum and RMSprop to updates parameters $W$ and $b$ regarding the hyperparameters $\beta_1$ and $\beta_2$ and the Momentum and RMSprop parameters $v_{dW}, v_{db}, S_{dW}, S_{db}$ with a EWMA bias correction so :

\begin{equation}
   \begin{cases}
   v_{dW} = \frac{v_{dW}}{1-\beta^t}, v_{db} = \frac{v_{db}}{1-\beta^t} \\
   S_{dW} = \frac{S_{dW}}{1-\beta^t}, S_{db} = \frac{S_{db}}{1-\beta^t}
   \end{cases}
\end{equation}

Leads to :

\begin{equation}
   \begin{cases}
       W = W - \alpha \frac{v_{dW}}{\sqrt{S_{dW}}+\epsilon} \\
       b = b - \alpha \frac{v_{db}}{\sqrt{S_{db}}+\epsilon}
   \end{cases}
\end{equation}

Where : 
- $\beta_1$ and $\beta_2$ are Adam's hyperparameters
- $\beta_1, \beta_2, \epsilon$ are usually set respectively to 0.9, 0.999 and 10e-8

In [ ]:
def initialize_adam(parameters:dict):
    """
    Initializes Adam parameters

    Parameters
    ----------
    parameters : dict
         Initializes v and s as two python dictionaries.
        
    Returns
    -------
    v : dict
        Will contain the exponentially weighted average of the gradient. Initialized with zeros.
    s : dict
        Will contain the exponentially weighted average of the squared gradient. Initialized with zeros.

    """
    L = len(parameters) // 2 # number of layers in the neural networks
    v = {}
    s = {}
    
    for l in range(1, L + 1):
        v["dW" + str(l)] = np.zeros(parameters["W" + str(l)].shape)
        v["db" + str(l)] = np.zeros(parameters["b" + str(l)].shape)
        s["dW" + str(l)] = np.zeros(parameters["W" + str(l)].shape)
        s["db" + str(l)] = np.zeros(parameters["b" + str(l)].shape)
        
    return v, s



def adam_update_parameters(parameters:dict, grads:dict, v:dict, s:dict, t:int, 
                           learning_rate:float=0.01, beta1:float=0.9, beta2:float=0.999, epsilon:float=1e-8):
    """
    Update parameters using Adam
    
    Parameters
    ----------
    parameters : dict
        Contains the parameters for the current layer l
    grads : dict
        Contains the grads output of the backward pass.
    v : dict
        Contains the exponentially weighted average of the gradient.
    s : dict
        Contains the exponentially weighted average of the squared gradient.
    t : int
        Adam variable, counts the number of taken steps
    learning_rate : float
        Learning_rate
    beta1 : float
        Exponential decay hyperparameter for the first moment estimates 
    beta2 : float
        Exponential decay hyperparameter for the second moment estimates 
    epsilon : float
        Hyperparameter preventing division by zero in Adam updates
        
    Returns
    -------
    parameters : dict
        Containing the parameters for the current layer l
    v : dict
        Will contain the exponentially weighted average of the gradient. Initialized with zeros.
    s : dict
        Will contain the exponentially weighted average of the squared gradient. Initialized with zeros.
    v_corrected : dict
        Blabla
    s_corrected : dict
        Blabla

    """ 
    L = len(parameters) // 2
    v_corrected = {}                         
    s_corrected = {}
    
    for l in range(1, L+1):
        # Exponentially moving average of the gradients
        v["dW" + str(l)] = beta1 * v["dW" + str(l)] + (1 - beta1) * grads["dW" + str(l)]
        v["db" + str(l)] = beta1 * v["db" + str(l)] + (1 - beta1) * grads["db" + str(l)]
        
        # Compute bias-corrected for first moment estimate
        v_corrected["dW" + str(l)] = v["dW" + str(l)] / (1 - pow(beta1, t))
        v_corrected["db" + str(l)] = v["db" + str(l)] / (1 - pow(beta1, t))
        
        # Exponentially moving average of the squared gradients
        s["dW" + str(l)] = beta2 * s["dW" + str(l)] + (1 - beta2) * pow(grads["dW" + str(l)], 2)
        s["db" + str(l)] = beta2 * s["db" + str(l)] + (1 - beta2) * pow(grads["db" + str(l)], 2)
        
        # Compute bias-corrected for second moment estimate
        s_corrected["dW" + str(l)] = s["dW" + str(l)] / (1 - pow(beta2, t))
        s_corrected["db" + str(l)] = s["db" + str(l)] / (1 - pow(beta2, t))
        
        # Update parameters
        parameters["W" + str(l)] = parameters["W" + str(l)] - learning_rate * v_corrected["dW" + str(l)] / (np.sqrt(s_corrected["dW" + str(l)]) + epsilon)
        parameters["b" + str(l)] = parameters["b" + str(l)] - learning_rate * v_corrected["db" + str(l)] / (np.sqrt(s_corrected["db" + str(l)]) + epsilon)
        
        return parameters, v, s, v_corrected, s_corrected


Let's now, build our new model function `NN_model_2` based on the previous model function `NN_model`, with respect to the regularization and optimization hyperparameters.

In [ ]:
def NN_model_2(X:np.ndarray, Y:np.ndarray, layers_dims:list, optimizer:str='gd', learning_rate:float=0.0075, 
               mini_batch_size:int=64, lambd:float=0., keep_prob:float=1., beta:float=0.9, beta1:float=0.9, 
               beta2:float=0.999, epsilon:float=1e-8, epochs:int=1000):
    """
    Implements a L-layer neural network: [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID.
    With potential regularization techniques and optimization techniques.

    Parameters
    ----------
    X : np.array
        Input features.
    Y : np.array of shape (1, m)
        True "label" vector (containing 0 if Pikachu, 1 if Rondoudou)
    layers_dims : list of length (number of layers + 1)
        Contain the input size and each layer size.
    optimizer : str, optional
        Optimization technique to use. The default is gradient descent 'gd'.
    learning_rate : float, optional
        Learning rate of the gradient descent update rule. The default is 0.0075.
    mini_batch_size : int, optional
        Size of the mini_batch.
    lambd : float, optional
        L2 regularization parameter. The default is 0.
    keep_prob : float, optional
        Dropout regularization parameter. The default is 1.
    beta : float, optional
        Momentum optimization parameter. The default is 0.9.
    beta1 : float, optional
        Adam optimization parameter. The default is 0.9.
    beta2 : float, optional
        Adam optimization parameter. The default is 0.999.
    epsilon : float, optional
        Adam optimization parameter. The default is 1e-8.
    epochs : int, optional
         Number of iterations of the optimization loop. The default is 1000.

    Returns
    -------
    parameters : dict
        Parameters learnt by the model.
    costs : list
        List of costs for each epoch.

    """
    costs = []
    t = 1
    m = X.shape[1]
    
    # Initialize parameters
    parameters = initialize_parameters(layers_dims)
    
    # Initialize parameters optimizer
    if optimizer == "gd":
        pass
    elif optimizer == "momentum":
        v = initialize_velocity(parameters)
    elif optimizer == "adam":
        v, s = initialize_adam(parameters)
    
    print(f"Training is starting with minibatch size = {mini_batch_size} ...")
    for epoch in range(1, epochs+1):
        # Chose a minibatch per epoch
        minibatches = random_mini_batches(X, Y, mini_batch_size)
        cost_total = 0
        
        # Training through minibatch
        for minibatch in minibatches:
            # Select a minibatch
            (minibatch_X, minibatch_Y) = minibatch 
        
            # Forward pass
            yhat, caches, listD = forward(minibatch_X, parameters, keep_prob)
            
            # Compute cost and add to the cost total
            if lambd == 0:
                cost = compute_cost(yhat, minibatch_Y, mini_batch_size)
            else:
                cost = compute_cost_L2regularization(yhat ,minibatch_Y, layers_dims, parameters, lambd, mini_batch_size)
            cost_total += cost
        
            # Backward pass
            grads = backward(yhat, minibatch_Y, caches, lambd, listD, keep_prob)
        
            # Update parameters
            if optimizer == "gd":
                parameters = update_parameters(parameters, grads, learning_rate)
            elif optimizer == "momentum":
                parameters, v = momentum_update_parameters(parameters, grads, v, beta, learning_rate)
            elif optimizer == "adam":
                parameters, v, s, _, _ = adam_update_parameters(parameters, grads, v, s, t, learning_rate, beta1, beta2, epsilon)
                t += 1
        # Compute average cost
        cost_avg = cost_total / m
            
        # Printing
        if epoch==1 or epoch % 100 == 0 or epoch == epochs: #- 1:
            print("{} ------- Cost after iteration {}: {:.4f}".format(
                datetime.datetime.now().strftime('%Y-%m-%d_%H.%M.%S'), 
                epoch, 
                cost_avg))
        if epoch % 100 == 0 or epoch == epochs:
            costs.append(cost_avg)
            
    return parameters, costs

In [ ]:
# Preprocessing
data_train_orig, label_train, data_val_orig, label_val, classes = load_data()

m_train_2 = data_train_orig.shape[0]
m_test_2 = data_val_orig.shape[0]

# Reshape the training and test examples 
data_train_flatten_2 = data_train_orig.reshape(m_train_2, -1).T
data_val_flatten_2 = data_val_orig.reshape(m_test_2, -1).T

# Standardize data to have feature values between 0 and 1.
data_train_2 = data_train_flatten_2/255.
data_val_2 = data_val_flatten_2/255.

label_train_2 = label_train.T
label_val_2 = label_val.T

print ("\n----------------------\n")
print ("data_train's shape: " + str(data_train_2.shape))
print ("data_val's shape: " + str(data_val_2.shape))
print ("\n----------------------\n")

n_x_2 = data_train_orig.shape[1]*data_train_orig.shape[2]*data_train_orig.shape[3]
n_y_2 = 1

Length pikachu dataset : 98
Length Rondoudou dataset : 76

----------------------

data_train's shape: (30000, 140)
data_val's shape: (30000, 34)

----------------------



In [ ]:
# Training a 4-layers model with Adam optimization
layers_dims_2 = [n_x_2, 20, 10, 5, n_y_2]
parameters, costs = NN_model_2(data_train_2,
                               label_train_2,
                               layers_dims_2,
                               optimizer = 'adam',
                               learning_rate = 0.001,
                               epochs = 1000)

pred_train = predict(data_train_2, label_train_2, parameters)
pred_test = predict(data_val_2, label_val_2, parameters, training_set=False)

Training is starting with minibatch size = 64 ...
2022-04-21_12.16.23 ------- Cost after iteration 1: 0.6721880481245561
2022-04-21_12.16.37 ------- Cost after iteration 100: 0.08118614168403186
2022-04-21_12.16.50 ------- Cost after iteration 200: 0.004229460522253018
2022-04-21_12.17.03 ------- Cost after iteration 300: 0.002257582105592734
2022-04-21_12.17.16 ------- Cost after iteration 400: 0.0013789982106878148
2022-04-21_12.17.29 ------- Cost after iteration 500: 0.0009743559452495649
2022-04-21_12.17.42 ------- Cost after iteration 600: 0.0007240775113913614
2022-04-21_12.17.55 ------- Cost after iteration 700: 0.0005558182328832589
2022-04-21_12.18.08 ------- Cost after iteration 800: 0.0004445863091180383
2022-04-21_12.18.21 ------- Cost after iteration 900: 0.0003515737431901355
2022-04-21_12.18.34 ------- Cost after iteration 1000: 0.00028292321242051475
Training Accuracy: 1.0
Validation Accuracy: 0.9411764705882353



## [6 - Conclusion](#table) <a class="anchor" id="conclusion"></a>



## [5.2.2 - Batch Normalization](#table) <a class="anchor" id="BN"></a>

A common way to speed-up the learning is to normalized mini-batches such as mean and variance of the pre-activation layer is respectively 0 and 1. It permits to keep the features at the same range and prevents covariate shift. Finally, the layer parameters are more independent.

Usually, it applies on the pre-activation layer $Z^{[l-1]}$ to train the next layer parameters faster :
$$ Z_{norm} = \frac{Z-\mu}{\sqrt{\sigma^{2} + \epsilon}}$$ 
Such as $\mu^{[l]}(z_{norm}^{(i)}) = 0$ and $\sigma^{[l]}(z_{norm}^{(i)}) = 1$


Where :
- $\mu = \frac{1}{m} \sum^{}_{i}z^{(i)}$ is the mean regarding each node $i$
- $\sigma^2 = \frac{1}{m}\sum^{}_{i}(z^{(i)} - \mu^{(i)})^2$ is the standard deviation regarding each node $i$
- $\epsilon$ a parameter sets to prevent zero division

Since we want to have control on the mean and the variance distributions of $Z_{norm}$ we define :

$$ \tilde{Z} = \beta_{1} Z_{norm} + \beta_{2} Z_{norm} $$

*Note : covariate shift occurs when the distribution of independent (input) variables can change over time while the labeling stays the same (ex : training on real dogs and then trying to detect cartoon dogs).*